In [1]:
from demo.demo_utils import Demo

demo = Demo()

demo.launch()

/opt/conda/envs/diffinv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.86it/s]


(0.0, 0.4) 10 0
image 38.768768310546875
